In [1]:
import xarray as xr
import xesmf as xe
import matplotlib.pyplot as plt
import glob
import os
import datetime

/N/u/gongg/Quartz/anaconda3/envs/xesmf/lib/python3.10/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


In [3]:
pattern_june = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/*_????-06-??_*.nc'
pattern_july = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/*_????-07-??_*.nc'
pattern_aug  = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/*_????-08-??_*.nc'

files_june = glob.glob(pattern_june)
files_july = glob.glob(pattern_july)
files_aug  = glob.glob(pattern_aug)

files_jja = sorted(files_june + files_july + files_aug)

ds_era = xr.open_mfdataset("/N/project/Zli_lab/gongg/ERA_data/JJA/prec_1998_08.nc")
test_ds_wrf = xr.open_dataset(files_jja[0])

regridder = xe.Regridder(test_ds_wrf, ds_era, method="bilinear", periodic=False)

In [8]:
chunk_size = 24
for i in range(10*92*24, 20*92*24, chunk_size):
    chunk = files_jja[i : i + chunk_size]
    
    ds_wrf = xr.open_mfdataset(chunk, combine='by_coords')

    ds_wrf_regrid = regridder(ds_wrf)

    basename = os.path.basename(chunk[0])               
    date_str = basename.split("_")[4]                  
    yyyymmdd = date_str.replace("-", "")                

    out_file = f'/N/project/Zli_lab/gongg/CONUS404_data/regrid_bl/{yyyymmdd}.nc'

    ds_wrf_regrid.to_netcdf(out_file)

    iteration = i // chunk_size  # 当前迭代次数
    if iteration % 100 == 0:
        print(datetime.datetime.now())

2025-03-24 00:06:09.355574
2025-03-24 00:21:52.951222
2025-03-24 00:36:23.684555
2025-03-24 00:50:27.073052
2025-03-24 01:04:43.053941
2025-03-24 01:17:57.007902
2025-03-24 01:30:38.579120
2025-03-24 01:42:49.818192
2025-03-24 01:54:46.875605


In [2]:
ds = xr.open_mfdataset('/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/*_2002-08-05_*.nc')

In [3]:
ds

<xarray.Dataset> Size: 144MB
Dimensions:      (Time: 24, south_north: 1015, west_east: 1367)
Coordinates:
  * Time         (Time) datetime64[ns] 192B 2002-08-05 ... 2002-08-05T23:00:00
    XLAT         (south_north, west_east) float32 6MB dask.array<chunksize=(882, 1188), meta=np.ndarray>
    XLONG        (south_north, west_east) float32 6MB dask.array<chunksize=(882, 1188), meta=np.ndarray>
    XTIME        (Time) datetime64[ns] 192B dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: south_north, west_east
Data variables:
    PREC_ACC_NC  (Time, south_north, west_east) float32 133MB dask.array<chunksize=(1, 882, 1188), meta=np.ndarray>
Attributes: (12/152)
    TITLE:                            OUTPUT FROM WRF V3.9.1.1 MODEL
    START_DATE:                      2002-07-01_00:00:00
    SIMULATION_START_DATE:           1979-10-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        1368
    SOUTH-NORTH_GRID_DIMENSION:      1016
    BOTTOM-TOP_GRID_DIMENSION:       51
    ...                              ...
    Source_List:                     ./wrf_constants_list_conus404_v20201005.csv
    Source_Code:                     make_conusii_2d.csh
    history:                         Sat Jun 29 18:53:58 2024: /glade/u/apps/...
    Project:                         USGS404
    Conventions:                     CF-1.0
    Usage:                           ncgen -k nc4 -o 3_layers_stag.nc 3_layer...